# 勞動力市場對股價行情影響研究

本篇主要觀察勞動力市場對市場行情有沒有影響

勞動力資訊大約可以分成以下幾種 (歡迎補充!)
- 就業人口數據 : 了解就業比例
- 薪資和工資數據 : 了解勞工薪資結構
- 職業結構和行業數據 : 加入行業分類
- 裁員和招聘數據 : 勞動市場變動 -> 缺工 or 不願意上工
- 勞動生產率數據 : 生產力水平

## 台灣就業市場

從[中華民國統計資訊網](https://nstatdb.dgbas.gov.tw/dgbasall/webMain.aspx?sys=100&funid=qryout&funid2=A040107010&outmode=8&ym=10801&ymt=11305&cycle=41&outkind=1&compmode=0&ratenm=%u7D71%u8A08%u503C&fldlst=111111111111&codlst0=100&compmode=0&rr=q10622x&&rdm=R177870)抓取勞動資訊並且處理



In [24]:
from datetime import datetime
import pandas as pd
import requests
import json
import re

In [39]:
def get_employment_info(start_year:str, end_year:str) -> pd.DataFrame:
    """Get employment info from 中華民國統計資訊網, and return a dataframe

    Args:
        start_year (str): (ex. 1978)
        end_year (str): (ex. 2024)

    Returns:
        A dataframe of employment info
    """
    
    url = f"https://nstatdb.dgbas.gov.tw/dgbasall/webMain.aspx?sdmx/a040107010/1+2+3+4+5+6+7+8+9+10+11+12.1..M.&startTime={start_year}-M1&endTime={end_year}-M5"
    
    
    res = requests.get(url).json()["data"]
    raw_data = res["dataSets"][0]["series"]
    index = res["structure"]["dimensions"]
    columns_value = index["series"][0]["values"]
    rows_value = index["observation"][0]["values"]
    
    # 加入 column index
    columns = []
    for column in columns_value:
        columns.append(column["name"])
    # 加入 row index
    rows = []
    for row in rows_value:
        date_str = re.sub(r'M(\d{1,2})', lambda x: f'{int(x.group(1)):02d}', row["id"])
        date_obj = datetime.strptime(date_str, "%Y-%m")
        rows.append(date_obj)
    # 加入 data
    table = []
    for i in list(raw_data.keys()):
        col_data = []
        for j in list(raw_data[i]["observations"].keys()):
            col_data.append(raw_data[i]["observations"][j][0])
        table.append(col_data)
    
    # 建立 dataframe
    df = pd.DataFrame(index=rows, columns=columns)
    for i, row in enumerate(table):
        for j, value in enumerate(row):
            df.iat[j, i] = value
    
    return df 

In [40]:
employment = get_employment_info("1978", "2024")
employment.head()

,總人口數(千人),15歲以上民間人口(千人),勞動力(千人),就業人數(千人),失業人數(千人),15歲以上民間人口占總人口數(%),勞動力占總人口數(%),勞動力參與率(%),就業人口占總人口數(%),就業人口占15歲以上民間人口(%),就業人口占勞動力(%),失業率(%)
1978-01-01,16832,10662,6242,6117,125,63.34,37.08,58.54,36.34,57.37,97.99,2.01
1978-02-01,16854,10663,6195,6081,114,63.27,36.76,58.1,36.08,57.03,98.16,1.84
1978-03-01,16884,10706,6175,6066,108,63.41,36.57,57.68,35.93,56.66,98.24,1.76
1978-04-01,16908,10717,6229,6131,98,63.39,36.84,58.12,36.26,57.2,98.42,1.58
1978-05-01,16930,10746,6261,6155,106,63.47,36.98,58.26,36.36,57.28,98.3,1.7
